In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train =pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
test = pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
df_test = test['comment_text']
df = train[['target','comment_text']]

In [ ]:
from sklearn.model_selection import train_test_split
X = df['comment_text']
y = df['target']
y = np.where(y>=0.5,1.,0.)
x_train, x_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=42)

In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(x_train.values.reshape(-1,1), y_train)
print(sorted(Counter(y_resampled).items()))

In [ ]:
X_resampled = X_resampled.reshape(-1)

In [ ]:
!python -m spacy download en

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
from tqdm.auto import tqdm
x_train_v= np.zeros((len(x_train),96))
x_test_v= np.zeros((len(x_test),96))

for i, text in enumerate(tqdm( nlp.pipe(X_resampled) , total =len(x_train) )):
    x_train_v[i,:]=text.vector
    
for i, text in enumerate(tqdm ( nlp.pipe(x_test) , total= len(x_test))):
    x_test_v[i,:]=text.vector
    

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
svc = LinearSVC()
svc.fit(x_train_v,y_train)

In [ ]:
from sklearn import metrics

def print_report(clf, x_test, y_test):
    y_pred = clf.predict(x_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(svc, x_test_v, y_test)

In [ ]:
reg = LogisticRegression()
reg.fit(x_train_v, y_train)

In [ ]:
print_report(reg, x_test_v, y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units = 100, kernel_initializer = 'uniform', activation = 'relu', 
                input_dim = x_train_v.shape[1]))
model.add(Dense(units = 50, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 25, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(x_train_v, y_train,batch_size=32,epochs=40,validation_data=(x_test_v,y_test))

In [ ]:
model.evaluate(x_test_v, y_test)

# try embeddings with ngram using cnn